In [1]:
import wandb
import numpy as np
from sklearn.metrics import confusion_matrix

# Initialize wandb API
api = wandb.Api()

# Define project and run ID
entity = "philinthesky"  # Your WandB username or team name
project_name = "Master Thesis"  # Change to your actual project name
run_id = "zd2j8s0l" # Get this from wandb UI (not the name, but the ID)

# Fetch the run
run = api.run(f"{entity}/{project_name}/{run_id}")

# Retrieve history as a pandas DataFrame
history = run.history()

# Extract stored predictions and true labels
all_preds = np.array(history["all_preds"]).flatten()
all_targets = np.array(history["all_targets"]).flatten()

# Compute confusion matrix
conf_matrix = confusion_matrix(all_targets[-1], all_preds[-1])

print("Confusion Matrix:\n", conf_matrix)


True


KeyError: 'all_preds'

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
idx_labels = ["Left_Hand", "Right_Hand", "Both_Feet", "Tongue"]
df_cm = pd.DataFrame(conf_matrix, index=idx_labels, columns = idx_labels)
sns.heatmap(df_cm, annot = True, fmt = "g")
sns.set(font_scale=1.2)
plt.title("Confussion Matrix (Shallow)")
plt.show()

In [ ]:

from statsmodels.tsa.stattools import adfuller
# Initialize wandb API
api = wandb.Api()

# Define the project and run ID
project_name = "Master Thesis"  # Replace with your project name
run_id = "zd2j8s0l"  # Replace with your specific run ID

# Fetch the run from the API
run = api.run(f"philinthesky/{project_name}/{run_id}")

# Retrieve the history of the run (this contains all logged metrics)
history = run.history()

# Assuming you logged the accuracies with keys "class_0_accuracy", "class_1_accuracy", etc.
# You can adjust the key names based on what you logged
class_accuracies = {f'class_{i}': history[f'class_{i}_accuracy'].tolist() for i in range(4)}


# If you want to see the accuracy for each class over the epochs
simple_dict = {}
idx_labels = ["Left_Hand", "Right_Hand", "Both_Feet", "Tongue"]
for number, (class_name, accuracies) in enumerate(class_accuracies.items()):
    adf_result = adfuller(accuracies[0:-1])
    print(f"{class_name}: {adf_result}")
    print(len(accuracies[0:-1]))
    simple_dict[idx_labels[number]] = accuracies[0:-1]
    #print(f'ADF Statistic: {adf_result[0]}')
    #print(f'p-value: {adf_result[1]}')
    #print(f"{class_name}: {accuracies}")
    #data = ???




In [ ]:
idx_labels = ["Left_Hand", "Right_Hand", "Both_Feet", "Tongue"]
for count, (class_name, accuracies) in enumerate(class_accuracies.items()):
    print(count)
     # Convert accuracies to a pandas Series for easier handling
    accuracies_series = pd.Series(accuracies)
    nan_indices = np.where(np.isnan(accuracies))[0]
    print(nan_indices)
    # Check for NaN values
    nan_values = accuracies_series.isna().sum()
    
    # Check for Inf values
    inf_values = np.isinf(accuracies_series).sum()
    
    print(f"{idx_labels[count]}: NaN values = {nan_values}, Inf values = {inf_values}")

    # Plot the accuracies (if no NaN or Inf values)
    if nan_values == 0 and inf_values == 0:
        plt.plot(accuracies[0:-1], label=idx_labels[count])
    plt.plot(accuracies, label=idx_labels[count])

plt.ylabel("Accuracies")
plt.xlabel("Epochs")
plt.title("Shallow")
plt.legend(fontsize = 8)
plt.show()    

In [ ]:
# Set the index labels (classes)
idx_labels = ["Left_Hand", "Right_Hand", "Both_Feet", "Tongue"]
# Adjust the row index to represent time steps (100 data points)
time_steps = [f"Time Step {i+1}" for i in range(200)]  # Creates a list of time steps from "Time Step 1" to "Time Step 100"

# Construct the DataFrame with 100 rows (time steps) and 4 columns (for each class)
df_simple = pd.DataFrame(simple_dict, index=time_steps)

# Check the DataFrame
print(df_simple.head(10))  # Print first 5 rows to check

In [ ]:

import numpy as np
import pandas as pd
from statsmodels.tsa.api import VAR

def select_lag_bic(data, max_lags=12):
    """
    Selects the optimal lag based on Bayesian Information Criterion (BIC).
    
    Parameters:
        data (pd.DataFrame): Time series data with multiple variables (columns).
        max_lags (int): Maximum number of lags to evaluate.
    
    Returns:
        int: Optimal lag length based on BIC.
    """
    model = VAR(data)
    bic_values = []
    
    for lag in range(1, max_lags + 1):
        result = model.fit(lag)
        bic_values.append((lag, result.bic))

    # Find the lag with the minimum BIC value
    optimal_lag = min(bic_values, key=lambda x: x[1])[0]

    return optimal_lag

# Example usage:
optimal_lag = select_lag_bic(df_simple)  # df_simple is your DataFrame
print(f"Optimal number of lags based on BIC: {optimal_lag}")


In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag = 1
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df


df_granger_causation_matrix = grangers_causation_matrix(df_simple, variables = df_simple.columns)  


In [ ]:
print(df_granger_causation_matrix)

In [ ]:
#df_granger_causation_matrix
sns.heatmap(df_granger_causation_matrix, annot = True)#, fmt = "g")
plt.title("Granger Causation Matrix (Conformer)")
plt.show()